In [4]:
# portrait batch logic
import argparse
import logging
import pickle
import re
import os

import boto3
import numpy as np
import pandas as pd
from tqdm import tqdm

from tensorflow.python.keras.models import Model, save_model, load_model
from deepmatch.layers import custom_objects

In [5]:
########################################
# 从s3同步数据
########################################
def sync_s3(file_name_list, s3_folder, local_folder):
    for f in file_name_list:
        print("file preparation: download src key {} to dst key {}".format(os.path.join(
            s3_folder, f), os.path.join(local_folder, f)))
        s3client.download_file(bucket, os.path.join(
            s3_folder, f), os.path.join(local_folder, f))


def write_to_s3(filename, bucket, key):
    with open(filename, 'rb') as f:  # Read in binary mode
        return s3client.upload_fileobj(f, bucket, key)


default_bucket = 'sagemaker-us-east-1-002224604296'
default_mk_region = '1'
level_1 = 'recommender-system-film-mk'

parser = argparse.ArgumentParser()
parser.add_argument('--bucket', type=str, default=default_bucket)
parser.add_argument('--mk-region', type=str, default=default_mk_region)

args, _ = parser.parse_known_args()
bucket = args.bucket
mk_region = args.mk_region

prefix = f"{level_1}/{mk_region}"

print("bucket={}".format(bucket))
print("prefix='{}'".format(prefix))

s3client = boto3.client('s3')
local_folder = 'info'
if not os.path.exists(local_folder):
    os.makedirs(local_folder)
# 行为数据加载
file_name_list = ['action.csv']
s3_folder = '{}/system/user-data/clean/latest'.format(prefix)
sync_s3(file_name_list, s3_folder, local_folder)
# youtubednn模型相关文件
file_name_list = ['user_embeddings.h5']
s3_folder = 'recommender-system-film-mk/1/model/recall/youtubednn/'
sync_s3(file_name_list, s3_folder, local_folder)
# ub数据加载
file_name_list = ['raw_embed_user_mapping.pickle', 'raw_embed_item_mapping.pickle']
s3_folder = '{}/feature/action/'.format(prefix)
sync_s3(file_name_list, s3_folder, local_folder)
# 用户画像数据加载
file_name_list = ['portrait.pickle']
s3_folder = '{}/feature/recommend-list/portrait'.format(prefix)
sync_s3(file_name_list, s3_folder, local_folder)
# 倒排列表的pickle文件
file_name_list = ['movie_id_movie_property_dict.pickle']
s3_folder = '{}/feature/content/inverted-list/'.format(prefix)
sync_s3(file_name_list, s3_folder, local_folder)

# 加载用户数据
user_click_records = {}
data_mk = pd.read_csv('info/action.csv',sep='\t')
for reviewerID, hist in tqdm(data_mk[(data_mk['label'] == 1) & (data_mk['programType'] == 1)].groupby('userid')):
    pos_list = hist['programId'].tolist()
    user_click_records[reviewerID] = pos_list
# 加载pickle文件
file_to_load = open("info/movie_id_movie_property_dict.pickle", "rb")
dict_id_content = pickle.load(file_to_load)
print("length of movie_id v.s. movie_property {}".format(len(dict_id_content)))

file_to_load = open("info/movie_id_movie_property_dict.pickle", "rb")
dict_id_content = pickle.load(file_to_load)

file_to_load = open("info/raw_embed_item_mapping.pickle", "rb")
dict_item_mapping = pickle.load(file_to_load)

file_to_load = open("info/raw_embed_user_mapping.pickle", "rb")
dict_user_mapping = pickle.load(file_to_load)

# 加载模型
user_embedding_model = load_model('info/user_embeddings.h5', custom_objects)

bucket=sagemaker-us-east-1-002224604296
prefix='recommender-system-film-mk/1'
file preparation: download src key recommender-system-film-mk/1/system/user-data/clean/latest/action.csv to dst key info/action.csv
file preparation: download src key recommender-system-film-mk/1/model/recall/youtubednn/user_embeddings.h5 to dst key info/user_embeddings.h5
file preparation: download src key recommender-system-film-mk/1/feature/action/raw_embed_user_mapping.pickle to dst key info/raw_embed_user_mapping.pickle
file preparation: download src key recommender-system-film-mk/1/feature/action/raw_embed_item_mapping.pickle to dst key info/raw_embed_item_mapping.pickle
file preparation: download src key recommender-system-film-mk/1/feature/recommend-list/portrait/portrait.pickle to dst key info/portrait.pickle
file preparation: download src key recommender-system-film-mk/1/feature/content/inverted-list/movie_id_movie_property_dict.pickle to dst key info/movie_id_movie_property_dict.pickle


100%|██████████| 1802/1802 [00:00<00:00, 7097.35it/s]


length of movie_id v.s. movie_property 33767


In [3]:
def update_portrait_under_a_property(mt_content, mt_up, ratio):
    # decay logic
    for k, v in mt_up.items():
        if k != 'recent':
            if mt_up[k]['mark'] != '1':
                mt_up[k]['score'] = mt_up[k]['score'] * ratio
            else:
                mt_up[k]['mark'] = '0'

    # update logic
    for ct in mt_content:
        if ct != None:
            if ct not in mt_up:
                mt_up[ct] = {}
                mt_up[ct]['mark'] = '1'
                mt_up[ct]['score'] = 1.0
            else:
                mt_up[ct]['mark'] = '1'
                mt_up[ct]['score'] = mt_up[ct]['score'] + 1.0

    # find large score
    for k, v in mt_up.items():
        # update large score and type
        if k != 'recent':
            if mt_up[k]['score'] >= mt_up['recent'][1] and k not in mt_up['recent'][0]:
                mt_up['recent'][0].append(k)
                mt_up['recent'][1] = mt_up[k]['score']
########################################
# 初始化用户画像
# 数据结构:
# 'language':{'xx':{'mark':,'score':},...'recent':['xx',score]}}
# 'embedding':{'review':xxx,'photo':xxx,'ub':xxx}
########################################
def initial_user_portrait(user_list):
    user_data_frame = {}
    popularity_method_list = ['category', 'director',
                              'actor', 'language']
    for user in user_list:
        current_user_portrait = {}
        for mt in popularity_method_list:
            current_user_portrait[mt] = {}
            current_user_portrait[mt]['recent'] = []
            current_user_portrait[mt]['recent'].append([])
            current_user_portrait[mt]['recent'].append(0.0)
        user_data_frame[str(user)] = current_user_portrait
#     col_name = ['user_id']+popularity_method_list
#     portrait_pddf = pd.DataFrame.from_dict(user_data_frame, orient='index',
#                            columns=col_name)
#     portrait_pddf = portrait_pddf.reset_index(drop=True)
#     portrait_pddf.to_csv('info/portrait.csv')
    # save pickle file
    file_name = 'info/portrait.pickle'
    output_file = open(file_name, 'wb')
    pickle.dump(user_data_frame, output_file)
    output_file.close()
    # # backup original file
    # file_name = 'info/user_portrait_{}_raw.pickle'.format(user_name)
    # output_file = open(file_name, 'wb')
    # pickle.dump(current_user_portrait, output_file)
    # output_file.close()

########################################
# 用户画像更新逻辑
# 数据结构:
# 'language':{'xx':{'mark':,'score':},...{'recent':['xx',score]}}
# 'embedding':{'review':xxx,'photo':xxx,'ub':xxx}
########################################
def update_user_portrait_with_one_click(current_user_portrait, current_read_item):
    
#     # load user portrait for testing
#     file_to_load = open("info/user_portrait_{}.pickle".format(user_name), "rb")
#     current_user_portrait = pickle.load(file_to_load)
#     print("load user portrait of the content is {}".format(current_user_portrait))

    # 用户兴趣衰减系数
    decay_ratio = 0.8

    popularity_method_list = ['category', 'director',
                              'actor', 'language']

    for mt in popularity_method_list:
        update_portrait_under_a_property(
            dict_id_content[current_read_item][mt], current_user_portrait[mt], decay_ratio)

#     print("updated user portrait {}".format(current_user_portrait))
#     # save pickle files
#     file_name = 'info/user_portrait_{}.pickle'.format(user_name)
#     output_file = open(file_name, 'wb')
#     pickle.dump(current_user_portrait, output_file)
#     output_file.close()


def update_user_embedding(user_id, input_item_list):


#     file_to_load = open("info/user_portrait_{}.pickle".format(user_id), "rb")
#     current_user_portrait = pickle.load(file_to_load)
#     print("load user portrait of {}, the content is {}".format(
#         user_name, current_user_portrait))

    # 映射用户的embedding
    # 构建适合模型的输入
    watch_list_len = 50
    map_input_item_list = np.array([[0]*watch_list_len])
    watch_len = len(input_item_list)
    map_user_id = dict_user_mapping[str(user_id)]
    for cnt, item in enumerate(input_item_list):
        if cnt < 50:
            map_input_item_list[0][cnt] = dict_item_mapping[str(item)]
    model_input = {}
    model_input['user_id'] = np.array([int(map_user_id)])
    model_input['hist_movie_id'] = map_input_item_list
    model_input['hist_len'] = np.array([watch_len])

    # 更新用户的embeddings
#     print("model input {}".format(model_input))
    updated_user_embs = user_embedding_model.predict(
        model_input, batch_size=2 ** 12)

#     current_user_portrait['ub_embed'] = updated_user_embs

#     print("update user embeddings {}".format(updated_user_embs))
    
    return updated_user_embs

In [5]:
file_to_load = open("info/portrait.pickle", "rb")
dict_user_portrait = pickle.load(file_to_load)
print("update user portrait for batch users")
for user_id, input_item_list in user_click_records.items():
    print("user id {} item list {}".format(user_id,input_item_list))
    for ci in input_item_list:
        update_user_portrait_with_one_click(dict_user_portrait[str(user_id)], str(ci))
    dict_user_portrait[str(user_id)]['ub_embeddding'] = update_user_embedding(user_id, input_item_list)

update user portrait for batch users
user id 10541 item list [1691829, 1691829, 1692211]
user id 10899 item list [1692053, 1692053]
user id 10945 item list [1692092, 1692092, 1691711, 1691711, 1691708, 1691708, 1692090]
user id 11283 item list [1690157, 1690157, 1363219, 1692625, 1692605, 1692605]
user id 12648 item list [1692248, 1692248]
user id 12968 item list [1692080, 1692080]
user id 13394 item list [1692053, 1692053, 1692045, 1692045]
user id 13396 item list [1690115, 1690115, 1690115, 1690113, 1690113, 1691695, 1691695, 1690114]
user id 13518 item list [1644863, 1644863]
user id 13554 item list [1692405, 1692405, 1692405, 1692405, 1692419, 1692419, 1692419]
user id 13557 item list [1688532, 1688532, 1688532, 1690118, 1690118, 1690118, 1688457, 1688457, 1688457, 1690135, 1690135, 1690134, 1690134, 1690117, 1690117, 1688456, 1688442, 1594867, 1594867, 1594864, 1594864, 1594868, 1594868, 1594868, 1594868, 1594855, 1594855, 1692306, 1694255, 1694255, 1694232, 1694232, 1694246, 1694

user id 161201003292 item list [1691244, 1691244]
user id 161201003636 item list [1692752, 1692752]
user id 161201004425 item list [1683402, 1683402, 1691249, 1691249, 1691238, 1686527, 1686527, 1691248, 1691248, 1689647, 1689647, 1689652, 1689652]
user id 161201007343 item list [1664051, 1664051]
user id 161201010053 item list [1681129, 1681129]
user id 161201010166 item list [1692328, 1692328]
user id 161201011100 item list [1692373, 1692373]
user id 161201015161 item list [1683402, 1683402]
user id 161201015173 item list [1690544, 1690544, 1690549, 1690549]
user id 161201015345 item list [1691240, 1691240, 1691241, 1691241, 1691241, 1691241, 1691239, 1691239, 1691252, 1691252, 1690555, 1690555, 1691237, 1691237, 1691237, 1691237, 1690546, 1690546, 1688341, 1688341, 1690547, 1690547]
user id 161201016333 item list [1691255, 1691255, 1672055, 1672055, 1685894, 1685894, 1692333, 1692333]
user id 161201016731 item list [1690547, 1690547]
user id 161201017201 item list [1691237, 1691237]

user id 170517005975 item list [1691237, 1691237]
user id 170517006444 item list [1689215, 1689215, 1689217, 1689217]
user id 170517006448 item list [1683403, 1683403]
user id 170517007071 item list [1692333, 1692333, 1692752, 1692752]
user id 170517007195 item list [1683402, 1683402]
user id 170517008328 item list [1683404, 1683404]
user id 170517008404 item list [1675913, 1675913, 1691246, 1681133, 1681133]
user id 170517008985 item list [1691256, 1691256, 1691255, 1691255, 1691255, 1691255, 1691255, 1683403, 1683403, 1691244, 1691244, 1691254, 1691254, 1691254, 1670858, 1670858, 1670858, 1670858, 1670857, 1670857, 1670851, 1670851, 1670851, 1670853, 1670853, 1649521, 1649521, 1647193, 1647193, 1691245, 1691245, 1688344, 1688344, 1688344, 1688344, 1686535, 1686535, 1686534, 1686534, 1686533, 1686533, 1689161, 1689161, 1689162, 1689162, 1689163, 1689163, 1691280, 1691280, 1683408, 1683408, 1659765, 1659765, 1692811, 1692811, 1692782, 1692782, 1692791, 1692791]
user id 170517009775 ite

user id 170522000094 item list [1692331, 1692331, 1692752, 1692752, 1692752]
user id 170522000672 item list [1690577, 1690577, 1691238, 1691238, 1690549, 1690549, 1692333, 1692333]
user id 170522000823 item list [1692332, 1692332, 1692332, 1692332, 1692331, 1692331]
user id 170522001320 item list [1683402, 1683402, 1683402, 1683403, 1683403, 1683406, 1683406]
user id 170522001452 item list [1692330, 1692330]
user id 170522003059 item list [1691249, 1691249, 1691249, 1691248, 1691248, 1691248, 1692332, 1692332]
user id 170522003269 item list [1691238, 1691238, 1692329, 1692329]
user id 170522003706 item list [1690544, 1690544, 1691239, 1691239]
user id 170522003852 item list [1692329, 1692329]
user id 170522006667 item list [1683402]
user id 170522007060 item list [1691238, 1691238]
user id 170522007463 item list [1691240, 1690577, 1690577, 1688341, 1688341, 1688341, 1691238, 1691238, 1688340, 1688340, 1691248, 1691248, 1691246, 1691246, 1691246, 1691246, 1692329, 1692328, 1673079]
user

user id 170624024831 item list [1691256, 1691256, 1691254, 1691254, 1659765, 1659765, 1692333, 1692333, 1692329, 1692329]
user id 170624024969 item list [1692333, 1692333, 1692752, 1692752]
user id 170624025179 item list [1691245, 1691245]
user id 170624025396 item list [1690552, 1690552, 1691241, 1691241, 1692331, 1692331]
user id 170624025620 item list [1669368, 1669368, 1672056, 1672056, 1692333, 1692333, 1692332, 1692332, 1692752, 1692752]
user id 170624026674 item list [1688344, 1688344, 1669369, 1669369, 1669369, 1669369, 1675912, 1675912, 1692333, 1692333, 1692332, 1692332, 1692331, 1692331, 1692330, 1692330]
user id 170624026987 item list [1688343, 1688343, 1690551, 1690551, 1690550, 1690550]
user id 170624027164 item list [1688345, 1688345, 1692329, 1692329]
user id 170624027317 item list [1691241, 1691241, 1691244, 1691244, 1681130, 1681130, 1675912, 1675912]
user id 170624028138 item list [1691238, 1691238, 1692329, 1692329, 1692329]
user id 170625001556 item list [1670872, 

...

user id 180627019531 item list [1691240, 1691240, 1692331, 1692331, 1692752, 1692752]
user id 180627019695 item list [1691249, 1691249, 1691246, 1691246]
user id 180627019935 item list [1679555, 1679555, 1688344, 1688344, 1681129, 1681129, 1681130, 1681130, 1681131, 1681131, 1673078, 1673078]
user id 180627020129 item list [1691241, 1691241, 1688918, 1688918, 1670858, 1670858, 1670853, 1670853, 1688919, 1688919]
user id 180627020901 item list [1691238, 1691238]
user id 180627021082 item list [1692405, 1692405]
user id 180627021447 item list [1691246, 1691246, 1692332, 1692332]
user id 180627021628 item list [1691249, 1691249]
user id 180627021868 item list [1691256, 1691256, 1691256, 1691256, 1691256, 1691256, 1691256, 1691256, 1691240, 1691240, 1691255, 1691255, 1688345, 1688345, 1688345, 1688345, 1672055, 1672055, 1672055, 1672055, 1691244, 1691244, 1691254, 1691254, 1691254, 1691254, 1691254, 1691254, 1669369, 1669369, 1685894, 1685894, 1670872, 1670872, 1664051, 1664051, 1672057, 1

user id 181010003438 item list [1691245, 1691245]
user id 181010003832 item list [1691249, 1691249, 1691249, 1691249, 1691249, 1691249, 1691241, 1691241, 1691241, 1691241, 1691239, 1691239, 1691239, 1691239, 1691237, 1691237, 1690546, 1690546, 1688341, 1688341, 1686533, 1686533, 1690557, 1690557, 1690559, 1690559, 1691238, 1691238, 1690549, 1690549, 1688343, 1688343, 1688342, 1688342, 1690578, 1690578, 1690580, 1690580, 1690580, 1690580, 1688340, 1688340, 1686536, 1686536, 1686536, 1686536, 1686536, 1686536, 1686536, 1686536, 1675913, 1675913, 1675913, 1675913, 1672051, 1672051, 1672051, 1672051, 1672051, 1672051, 1672051, 1690551, 1690551, 1681134, 1681134, 1681134, 1681134, 1681134, 1681134, 1681134, 1691248, 1691248, 1691248, 1691248, 1690550, 1690550, 1690560, 1690560, 1690576, 1690576, 1691247, 1691247, 1691247, 1691247, 1672057, 1672057, 1686537, 1686537, 1686537, 1686537, 1681132, 1681132, 1681133, 1681133, 1681133, 1681133, 1675912, 1675912, 1688339, 1688339, 1675914, 1675914, 

user id 181012000595 item list [1681129, 1681129, 1690550, 1690550]
user id 181012000627 item list [1691241, 1691239, 1691239]
user id 181012000636 item list [1692419]
user id 181012000661 item list [1691254, 1691254, 1692333, 1692333]
user id 181012000727 item list [1692408, 1692408]
user id 181012000878 item list [1692333, 1692333, 1692752]
user id 181012000881 item list [1691239, 1691239, 1691238, 1691238]
user id 181012000929 item list [1691237, 1691237, 1690543, 1692328, 1692328]
user id 181012000937 item list [1692373, 1692373]
user id 181012000973 item list [1691256, 1691256, 1672055, 1692333, 1692333, 1692752]
user id 181012001024 item list [1691239, 1691239, 1691239, 1691239, 1691238, 1691238, 1692329, 1692329, 1692328, 1692328]
user id 181012001101 item list [1692331, 1692331]
user id 181012001193 item list [1691240, 1691240]
user id 181012001196 item list [1692373, 1692373, 1692366, 1692366, 1692366, 1692366, 1692365, 1692365, 1692368, 1692368]
user id 181012001294 item list

user id 181109008924 item list [1670123, 1670123, 1692291]
user id 181109009274 item list [1670121, 1670121]
user id 181113003550 item list [1681277, 1681277, 1692202, 1692202]
user id 181113005242 item list [1670123, 1670123]
user id 181113005725 item list [1670123, 1670123, 1670121, 1670121, 1669802, 1669802]
user id 181113008593 item list [1670123, 1670123, 1670122, 1670122]
user id 181113010347 item list [1692293, 1692293, 1692282, 1692282]
user id 181208021281 item list [1690118, 1690118]
user id 181208021287 item list [1466940, 1466940, 1694286, 1694286]
user id 181208021294 item list [1690135, 1690135, 1690132, 1690132]
user id 181208021441 item list [1466937, 1466937, 1466943, 1466943, 1466944, 1466944]
user id 181208021453 item list [1692053, 1692044]
user id 181208021456 item list [1694298, 1694298]
user id 181208021490 item list [1692202, 1692202]
user id 181208021630 item list [1694272, 1694272]
user id 181210001845 item list [1692034]
user id 181210004160 item list [169207

user id 21009 item list [1688457, 1688457]
user id 21013 item list [1363219, 1363219]
user id 21084 item list [1644873, 1644873]
user id 21089 item list [1466937, 1466937, 1466939]
user id 21140 item list [1645590, 1645590]
user id 21164 item list [1688531, 1688531, 1688516, 1688516]
user id 21174 item list [1466928, 1466928]
user id 21180 item list [1692226, 1692226]
user id 21185 item list [1692226, 1692226, 1644875, 1644875]
user id 21214 item list [1644866, 1644866]
user id 21224 item list [1688532, 1688457, 1688457]
user id 21256 item list [1690177, 1690177, 1688456, 1688456]
user id 21258 item list [1690177, 1690177]
user id 21274 item list [1645590, 1645590]
user id 21286 item list [1692102]
user id 21355 item list [1692201, 1692201]
user id 21411 item list [1692053]
user id 21429 item list [1693875, 1693875, 1693874, 1693874]
user id 21435 item list [1363238, 1363238]
user id 21479 item list [1644874, 1644874]
user id 21499 item list [1645589, 1645589, 1645588, 1645588]
user id

In [7]:
file_to_load = open("info/portrait.pickle", "rb")
dict_user_portrait = pickle.load(file_to_load)
dict_user_portrait['13518']

{'category': {'recent': [['comedy', 'drama', 'romance'], 1.0],
  'comedy': {'mark': '1', 'score': 6.0},
  'drama': {'mark': '1', 'score': 6.0},
  'romance': {'mark': '1', 'score': 6.0}},
 'director': {'recent': [['tucky williams'], 1.0],
  'tucky williams': {'mark': '1', 'score': 6.0}},
 'actor': {'recent': [['tucky williams',
    'maya jamner',
    'roni jonah',
    'l.e. barone',
    'amanda k. morales',
    'kayla perkins',
    'vanessa allen',
    'macy allen',
    'dennii cormney'],
   1.0],
  'tucky williams': {'mark': '1', 'score': 6.0},
  'maya jamner': {'mark': '1', 'score': 6.0},
  'roni jonah': {'mark': '1', 'score': 6.0},
  'l.e. barone': {'mark': '1', 'score': 6.0},
  'amanda k. morales': {'mark': '1', 'score': 6.0},
  'kayla perkins': {'mark': '1', 'score': 6.0},
  'vanessa allen': {'mark': '1', 'score': 6.0},
  'macy allen': {'mark': '1', 'score': 6.0},
  'dennii cormney': {'mark': '1', 'score': 6.0}},
 'language': {'recent': [['english'], 1.0],
  'english': {'mark': '1'

In [6]:
# 存储和更新用户画像

file_name = 'info/portrait.pickle'
output_file = open(file_name, 'wb')
pickle.dump(dict_user_portrait, output_file)
output_file.close()
write_to_s3(file_name, bucket, "{}/feature/recommend-list/portrait/{}".format(prefix,file_name.split('/')[-1]))

In [7]:
!python portrait-batch.py


bucket=sagemaker-us-east-1-002224604296
prefix='recommender-system-film-mk/1'
file preparation: download src key recommender-system-film-mk/1/system/user-data/clean/latest/action.csv to dst key info/action.csv
DeepCTR version 0.8.5 detected. Your version is 0.8.4.
Use `pip install -U deepctr` to upgrade.Changelog: https://github.com/shenweichen/DeepCTR/releases/tag/v0.8.5
file preparation: download src key recommender-system-film-mk/1/model/recall/youtubednn/user_embeddings.h5 to dst key info/user_embeddings.h5
file preparation: download src key recommender-system-film-mk/1/feature/action/raw_embed_user_mapping.pickle to dst key info/raw_embed_user_mapping.pickle
file preparation: download src key recommender-system-film-mk/1/feature/action/raw_embed_item_mapping.pickle to dst key info/raw_embed_item_mapping.pickle
file preparation: download src key recommender-system-film-mk/1/feature/recommend-list/portrait/portrait.pickle to dst key info/portrait.pickle
file preparation: download s

user id 161121004063 item list [1692329, 1692329]
user id 161121004404 item list [1691237, 1691237, 1691238, 1692329, 1692329]
user id 161121004940 item list [1691239, 1691238]
user id 161121005822 item list [1692329, 1692329]
user id 161121006038 item list [1691237, 1691237, 1692329, 1692329]
user id 161121006107 item list [1692328, 1692328]
user id 161121007235 item list [1688342, 1688342]
user id 161121008233 item list [1689209, 1689209, 1689164, 1689164, 1692345, 1692345]
user id 161121008330 item list [1692340, 1692340, 1692340, 1692339, 1692339, 1692337, 1692337]
user id 161121009030 item list [1683403, 1683403]
user id 161121009161 item list [1683402, 1683402]
user id 161121009238 item list [1683405, 1683405]
user id 161121009394 item list [1691252, 1691252]
user id 161121009569 item list [1691255, 1691255, 1691254, 1691254, 1662524, 1662524, 1692752, 1692752]
user id 161121011455 item list [1672057, 1672057]
user id 161121011593 item list [1683403, 1683403, 1691241, 1691241, 16

user id 17030 item list [1644874, 1644874]
user id 170502004991 item list [1692333, 1692333, 1692752, 1692752, 1692752]
user id 170502005825 item list [1690543, 1690543]
user id 170502007092 item list [1675914, 1675914, 1692342, 1692342]
user id 170502007395 item list [1690551, 1690551]
user id 170502007738 item list [1691255, 1691255, 1690543, 1690543]
user id 170502007747 item list [1688345, 1688345]
user id 170502007876 item list [1692332, 1692332, 1692332, 1692332, 1692331, 1692331]
user id 170502008087 item list [1692329, 1692329]
user id 170502012021 item list [1692344, 1692344]
user id 170502013613 item list [1683402, 1683402]
user id 170502014490 item list [1686537, 1686537]
user id 170502016106 item list [1691252, 1691252, 1691251, 1691251, 1690556, 1690556, 1690554, 1690554, 1690558, 1690558]
user id 170502016307 item list [1692752, 1692752]
user id 170502016591 item list [1692329, 1692329]
user id 170502016684 item list [1690546, 1690546, 1690545, 1690545, 1690550, 1690550, 

user id 170518015903 item list [1690544, 1690544, 1691241, 1691241, 1691239, 1691239, 1691239, 1691239, 1691239, 1691237, 1691237, 1690549, 1690549, 1690545, 1690545, 1689215, 1689215]
user id 170518019114 item list [1692329, 1692329]
user id 170518019266 item list [1683402, 1683402, 1691237, 1691237, 1688343, 1688343, 1690548, 1690548, 1692329, 1692329]
user id 170518022117 item list [1670858, 1670858]
user id 170518024977 item list [1692329, 1692329]
user id 170518025352 item list [1664051, 1664051, 1672057, 1672057, 1672057, 1669368, 1669368, 1692333, 1692333, 1692752, 1692752]
user id 170518025644 item list [1690549, 1690549]
user id 170518025882 item list [1686536, 1686536, 1692329, 1692329, 1692342, 1692342, 1692342]
user id 170518026477 item list [1688917, 1688917, 1688919, 1688919, 1688922, 1688922, 1688923, 1688923, 1681880, 1681880, 1681882, 1681882, 1681883, 1681883, 1678847, 1678847]
user id 170518026818 item list [1690552, 1690552, 1691240, 1691240, 1690546, 1688341, 16883

user id 170624000717 item list [1691249, 1691249]
user id 170624000802 item list [1690552, 1690552, 1688345, 1688345, 1692333, 1692333, 1692333, 1692333]
user id 170624001201 item list [1675912, 1675912]
user id 170624001929 item list [1691255, 1691255, 1691255, 1692338, 1692338]
user id 170624002243 item list [1688919, 1688919, 1692373, 1692373, 1692366, 1692366]
user id 170624002689 item list [1692339, 1692339, 1692341, 1692341]
user id 170624004608 item list [1691241, 1691241, 1691238, 1691238]
user id 170624005978 item list [1683402, 1683402]
user id 170624006806 item list [1692329, 1692329, 1692328, 1692328]
user id 170624007374 item list [1691237, 1691237, 1692811, 1692811]
user id 170624007513 item list [1692367, 1692367]
user id 170624007870 item list [1683402, 1683402, 1683403, 1683403]
user id 170624008518 item list [1691244, 1691244, 1689207, 1689207]
user id 170624010486 item list [1683404, 1683404]
user id 170624010503 item list [1688345, 1688345, 1688345, 1688345, 1688345

...

user id 180628015944 item list [1691240, 1691240, 1691240, 1691240, 1688342, 1688342]
user id 180629017802 item list [1691244, 1691244]
user id 180630006000 item list [1691239, 1691239, 1691238, 1691238]
user id 180630010312 item list [1686547, 1686547, 1686542, 1686542]
user id 180704011267 item list [1692329]
user id 180713023030 item list [1694272, 1694272]
user id 180713023145 item list [1676922, 1676922, 1692202, 1692202, 1694250, 1694250]
user id 180713023422 item list [1692076, 1692076, 1691706, 1691706]
user id 180713023822 item list [1670122, 1670122, 1694928, 1694928, 1694944, 1694944, 1694943, 1694943, 1694927, 1694927]
user id 180713023918 item list [1694267, 1694267]
user id 180713024528 item list [1694255, 1694255]
user id 180713024968 item list [1692053, 1692053, 1692115, 1692115, 1692115, 1692115, 1692101, 1692101]
user id 180713024983 item list [1363219, 1363219]
user id 180910004493 item list [1690117, 1690117, 1692290, 1692290, 1692290, 1692290, 1692291, 1692291, 169

user id 181011006192 item list [1692419, 1692419]
user id 181011007358 item list [1685894, 1685894, 1692333, 1692333]
user id 181011007762 item list [1691280, 1691280]
user id 181011008816 item list [1692329, 1692329]
user id 181011009543 item list [1691239, 1691239, 1688341, 1690543, 1690543, 1688340, 1688340, 1688340, 1688340, 1690547, 1690547, 1692329, 1692329]
user id 181011009662 item list [1686553, 1686553]
user id 181011009950 item list [1692329, 1692329, 1692328, 1692328]
user id 181011009966 item list [1691244, 1691244, 1675912, 1675912]
user id 181011010077 item list [1691240, 1691240, 1691240, 1691240, 1691240, 1690544, 1690544, 1691239, 1691239, 1690546, 1690546, 1688343, 1688343, 1688338, 1688338, 1690551, 1690551, 1690550, 1690550, 1690550, 1690550]
user id 181011010103 item list [1690543, 1690543, 1686533, 1686533, 1691250, 1691250, 1681129, 1681129, 1681130, 1681130]
user id 181011010112 item list [1691256, 1691256, 1691256, 1691256, 1690552, 1672055, 1672055, 1691249, 

user id 181013003165 item list [1691245, 1691245, 1692330, 1692330]
user id 181013003489 item list [1691240, 1691240, 1691241, 1691241, 1691239, 1691239, 1691239, 1691238, 1691238]
user id 181013003685 item list [1690552, 1690552, 1688345, 1688345, 1688345, 1688345, 1670872, 1670872, 1670872, 1670872, 1670872, 1664051, 1664051, 1664051, 1669368, 1669368, 1672056, 1672056, 1672056, 1672056, 1689168, 1689168, 1692333, 1692333, 1692353, 1692353, 1692345, 1692345, 1692350, 1692350, 1692752, 1692752, 1692752, 1692752, 1692752, 1692752, 1692347, 1692347]
user id 181013003928 item list [1691240, 1691240, 1691240, 1691240, 1691249, 1691249, 1691249, 1691249, 1691241, 1691241, 1691239, 1691239, 1691237, 1691237, 1690546, 1690546, 1686535, 1686535, 1686534, 1686534, 1691238, 1691238, 1690580, 1690580, 1690547, 1690547, 1691247, 1691247, 1691247, 1691247, 1690579, 1690579, 1692329, 1692329, 1692329, 1692329, 1692329, 1692329, 1692329, 1692329, 1692329, 1692373, 1692373, 1692328, 1692328, 1692328,

user id 19587 item list [1691281, 1691281, 1670121, 1670121]
user id 19612 item list [1691568, 1691568]
user id 19626 item list [1466939]
user id 19640 item list [1690155, 1690155]
user id 19669 item list [1691354, 1691354]
user id 19690 item list [1644866, 1644866, 1691018, 1691018]
user id 19691 item list [1688455, 1688455]
user id 19715 item list [1690212, 1690212]
user id 19724 item list [1690188, 1690188]
user id 19725 item list [1645589, 1645589]
user id 19762 item list [1691512, 1691517, 1691517]
user id 19771 item list [1690155, 1690155, 1644874, 1644874]
user id 19815 item list [1644864, 1644864]
user id 19821 item list [1692047, 1692047]
user id 19834 item list [1690157, 1690157]
user id 20027 item list [1644869, 1644869]
user id 200312001122 item list [1670123, 1670123, 1670122, 1670122]
user id 200312001996 item list [1692474, 1692474, 1694291, 1694291]
user id 200617006851 item list [1647197, 1647197]
user id 200617012067 item list [1692202, 1692202]
user id 200617014014 i

user id 2424 item list [1692419, 1692419, 1692406, 1692406]
user id 24244 item list [1690112, 1690112]
user id 24331 item list [1645590, 1645590, 1690138]
user id 24384 item list [1690115, 1690115]
user id 24403 item list [1639643, 1639643, 1693877, 1693877]
user id 24408 item list [1690177]
user id 24415 item list [1644864, 1644864]
user id 24422 item list [1471792, 1471792]
user id 24436 item list [1645589, 1645589]
user id 24437 item list [1693878, 1693878, 1693879, 1693879]
user id 24466 item list [1694249, 1694249, 1694250, 1694250, 1694251]
user id 24539 item list [1644873, 1644873]
user id 24569 item list [1645588]
user id 24591 item list [1645589, 1645589]
user id 24609 item list [1466938, 1466938, 1644875, 1644875]
user id 24659 item list [1690157, 1690157]
user id 24673 item list [1644867, 1644867]
user id 24689 item list [1690144, 1690144]
user id 24717 item list [1644874, 1644874]
user id 24787 item list [1690135, 1690135]
user id 24811 item list [1690157, 1690157]
user id 